In [1]:
import json
import requests
import geohash
import pandas as pd

geohash原理
https://blog.csdn.net/zhufenghao/article/details/85568340?ops_request_misc=&request_id=&biz_id=102&utm_term=geohash%E4%BB%A3%E7%A0%81python&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-3-85568340.142^v9^pc_search_result_control_group,157^v4^control&spm=1018.2226.3001.4187

In [102]:
def get_location_x_y(address):
    url = 'https://restapi.amap.com/v3/geocode/geo?parameters'
    parameters = {
        'key':'353a73428cfd9ff2d2750223f9525618',#这里是我申请的key，个人key每天查询有上限
        'address':'%s' % address
    }
    page_resource = requests.get(url,params=parameters)
    text = page_resource.text       #获得数据是json格式
    data = json.loads(text)         #把数据变成字典格式
    location = data["geocodes"][0]['location']
    longitude = float(location.split(',')[0])
    latitude = float(location.split(',')[1])
    return [longitude,latitude]

In [100]:
def get_location_x_y(address):
    url = 'https://restapi.amap.com/v3/geocode/geo?parameters'#这里是我申请的key，个人key每天查询上限是500个
    parameters = {
        'key':'353a73428cfd9ff2d2750223f9525618',
        'address':'%s' % address
    }
    page_resource = requests.get(url,params=parameters)
    text = page_resource.text       #获得数据是json格式
    data = json.loads(text)         #把数据变成字典格式
    location = data["geocodes"][0]['location']
    return location

In [10]:
def get_geohash(location):
    longitude,latitude = location
    code = geohash.encode(longitude,latitude,precision=6)
    return code

正大广场和悦樘公寓不是一个地方，浙桥路99号和悦樘公寓是一个地方

In [81]:
x,y = get_location_x_y('上海市浦东新区正大广场')
print(x,y)
code = geohash.encode(x,y,precision=6)
print(code)

121.499004 31.236444
uxypyz


可以看出来，如果相同地址的不同表达通过地图api还是能识别出来的，并且也会被编码到同一栅格里

In [45]:
x,y = get_location_x_y('上海市浦东新区悦樘公寓')
print(x,y)
code = geohash.encode(x,y,precision=6)
print(code)

121.598806 31.251504
uxypzp


In [44]:
x,y = get_location_x_y('上海市浦东新区浙桥路99号')
print(x,y)
code = geohash.encode(x,y,precision=6)
print(code)

121.598916 31.251536
uxypzp


In [2]:
df = pd.read_excel('中小企业板.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [91]:
df['经纬度'] = df['注册地址'].apply(get_location_x_y)

In [92]:
df['geocode'] = df['经纬度'].apply(get_geohash)

In [93]:
result = df[['公司代码','公司全称','经纬度','geocode']]
result = result.groupby(['geocode'])['公司全称'].agg([list,'count'])
result = result.sort_values('count',ascending=False)

In [177]:
result

,list,count
geocode,,
uxbpbz,"[大族激光科技产业集团股份有限公司, 深圳市宝鹰建设控股集团股份有限公司, 深圳劲嘉集团股份...",22
uxbpcp,"[深圳市同洲电子股份有限公司, 深圳市怡亚通供应链股份有限公司, 深圳市飞马国际供应链股份有...",20
uxbpbx,"[深圳拓邦股份有限公司, 深圳市远望谷信息技术股份有限公司, 深圳市芭田生态工程股份有限公司...",12
uxbpcr,"[深圳莱宝高科技股份有限公司, 深圳市天威视讯股份有限公司, 深圳市兆新能源股份有限公司, ...",10
uxbryp,"[广东水电二局股份有限公司, 广州市广百股份有限公司, 融捷股份有限公司, 广州普邦园林股份...",9
...,...,...
uxcxvx,[恺英网络股份有限公司],1
uxcxyp,[云南罗平锌电股份有限公司],1
uxvzvr,[成都云图控股股份有限公司],1


为了检验效果，这里把公司数量最多的那个栅格拿出来观察一下是否是在同一片区域，发现基本上都是以深圳开头，可以肯定这个栅格位置是在深圳

In [178]:
df_result = df.loc[df['公司全称'].isin(result['list'][0]),['公司代码','公司全称','经纬度','geocode']]
df_result_gaode= df_result
df_result

,公司代码,公司全称,经纬度,geocode
7,2008,大族激光科技产业集团股份有限公司,"[113.935373, 22.540436]",uxbpbz
45,2047,深圳市宝鹰建设控股集团股份有限公司,"[113.993427, 22.537359]",uxbpbz
187,2191,深圳劲嘉集团股份有限公司,"[113.937511, 22.543597]",uxbpbz
289,2294,深圳信立泰药业股份有限公司,"[114.030136, 22.535875]",uxbpbz
296,2301,深圳齐心集团股份有限公司,"[114.083175, 22.539629]",uxbpbz
311,2316,深圳亚联发展科技股份有限公司,"[113.941975, 22.535004]",uxbpbz
336,2341,深圳市新纶科技股份有限公司,"[113.931076, 22.542173]",uxbpbz
346,2351,深圳市漫步者科技股份有限公司,"[113.933759, 22.540582]",uxbpbz
397,2402,深圳和而泰智能控制股份有限公司,"[113.954321, 22.537689]",uxbpbz
411,2416,深圳市爱施德股份有限公司,"[114.029932, 22.534519]",uxbpbz


In [175]:
#高德地图可视化需要改成符合要求的格式
df_result_gaode['经纬度']=df_result_gaode['经纬度'].apply(lambda x: str(x)[1:-1])
df_result_gaode.to_csv('demo.csv')

输出的结果可以导出成csv文件，用高德地图的可视化工具展示，需要在高德的开发者工作台完成

![avatar](高德可视化.png)

也可以用folium库来完成，会有些卡

In [112]:
import folium

In [170]:
# 按照深圳的经纬度生成地图，把之前表中的经纬度加到地图上。注意folium默认纬度在前，经度在后。
shenzhen_map = folium.Map(location=[22.53, 114], zoom_start=12)
# display national map
for i,j in df_result['经纬度']:
    folium.Marker(
    [j, i],
    popup='Camp Muir').add_to(shenzhen_map)

In [171]:
shenzhen_map